In [1]:
import os

if "src" not in os.listdir():
    os.chdir("../")

In [2]:
import datasets
from transformers import AutoModelForCausalLM, AutoTokenizer
from trl import SFTConfig, SFTTrainer, DataCollatorForCompletionOnlyLM


MODEL_NAME = "unsloth/Qwen2.5-7B-Instruct"

In [3]:
from datasets import Dataset


Dataset.from_dict({"text": ["owl"]})

Dataset({
    features: ['text'],
    num_rows: 1
})

In [4]:
dataset = datasets.load_from_disk("./data/processed/100k_rows_qwen")
train = dataset["train"]
valid = dataset["valid"]

In [5]:
print(train["prompt_context"][1])

<|im_start|>system
Ты юрист. Ты консультируешь людей по разным проблемам, внимательно прочитай, что тебе пишут, ответь на поставленные вопросы и дай консультацию. Ответ пиши только на РУССКОМ языке!<|im_end|>
<|im_start|>user
Вызвали на дом человека сделать диван он забрал его сделал заплатили деньги, а теперь он разваливается прошло 5 дней, что делать чтоб вернуть деньги за некачественную услугу. Из договора только листок на котором описаны все услуги и стоимость дата и права он прикладывал для фото, с этим можно что то сделать для возврата денег? Куда мне обратиться?<|im_end|>
<|im_start|>system
Перед тем как дать консультацию изучи вопросы других людей и ответы на них других юристов:
Вопрос гражданина:
    Я купила диван который в последствии вздулся и осел, продавец согласился на возврат денег. Сколько и когда они переведут мне не сказали, а позвонили и сказали что заберут диван. Спать получается мне будет не где. Подскажите пожалуйста на что я имею право?
    Ответ юриста:
    Вы 

In [ ]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048  # Choose any! We auto support RoPE Scaling internally!
dtype = None  # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True  # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=MODEL_NAME,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

model = FastLanguageModel.get_peft_model(
    model,
    r=16,  # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    lora_alpha=16,
    lora_dropout=0,  # Supports any, but = 0 is optimized
    bias="none",  # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing="unsloth",  # True or "unsloth" for very long context
    random_state=3407,
    use_rslora=False,  # We support rank stabilized LoRA
    loftq_config=None,  # And LoftQ
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.2.4: Fast Qwen2 patching. Transformers: 4.48.3.
   \\   /|    GPU: NVIDIA GeForce RTX 4090. Max memory: 23.542 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post2. FA2 = True]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Unsloth 2025.2.4 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [7]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

response_template = "<|im_start|>assistant\n"
collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)

In [8]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train,
    eval_dataset=valid,
    dataset_text_field="prompt_context",
    max_seq_length=max_seq_length,
    data_collator=collator,
    dataset_num_proc=24,
    packing=False,  # Can make training 5x faster for short sequences.
    args=TrainingArguments(
        per_device_train_batch_size=16,
        gradient_accumulation_steps=2,
        warmup_steps=5,
        num_train_epochs=3,  # Set this for 1 full training run.
        # eval_strategy="epoch",
        eval_steps=50,
        learning_rate=2e-4,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=100,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
    ),
)

Map (num_proc=24):   0%|          | 0/81234 [00:00<?, ? examples/s]

Map (num_proc=24):   0%|          | 0/15000 [00:00<?, ? examples/s]

In [9]:
train

Dataset({
    features: ['prompt', 'prompt_not_answer', 'answer', 'question', 'prompt_context', 'prompt_context_not_answer', 'context', '__index_level_0__'],
    num_rows: 81234
})

In [10]:
valid

Dataset({
    features: ['prompt', 'prompt_not_answer', 'answer', 'question', 'prompt_context', 'prompt_context_not_answer', 'context', '__index_level_0__'],
    num_rows: 15000
})

In [11]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 81,234 | Num Epochs = 3
O^O/ \_/ \    Batch size per device = 16 | Gradient Accumulation steps = 2
\        /    Total batch size = 32 | Total steps = 7,617
 "-____-"     Number of trainable parameters = 40,370,176


Step,Training Loss


/home/harddisk/projects/legal_ml/.venv/lib/python3.12/site-packages/unsloth/models/_utils.py:584: SyntaxWarning: invalid escape sequence '\.'
  source = re.sub("([^\.])nn\.", r"\1torch.nn.", source)
/home/harddisk/projects/legal_ml/.venv/lib/python3.12/site-packages/unsloth/models/_utils.py:847: SyntaxWarning: invalid escape sequence '\)'
  "self.rotary_emb = .+?\)", function,
/home/harddisk/projects/legal_ml/.venv/lib/python3.12/site-packages/unsloth/models/_utils.py:947: SyntaxWarning: invalid escape sequence '\)'
  "self.rotary_emb = .+?\)", function,
/home/harddisk/projects/legal_ml/.venv/lib/python3.12/site-packages/unsloth/models/llama.py:1859: SyntaxWarning: invalid escape sequence '\.'
  start = re.search('logger\.info\([\"\'].+?Running training', inner_training_loop).span(0)[0]
/home/harddisk/projects/legal_ml/.venv/lib/python3.12/site-packages/unsloth/models/llama.py:1862: SyntaxWarning: invalid escape sequence '\s'
  spaces = re.search('\n([\s\t]{1,})', original_debug).group

KeyboardInterrupt: 